### 앙상블 학습
- 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘
- 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘

정형데이터 : 어떤 구조로 정리된 파일 ex) CSV파일, 엑셀, 데이터베이스   
비정형데이터: 데이터베이스나 엑셀로 표현하기 어려운 것들 ex) 텍스트 데이터, 디지털카메라로 찍은 사진, 디지털 음악

#### 트리의 앙상블 - 랜덤 포레스트
- 앙상블 학습의 대표 주자 (안정적인 성능)
- 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만든다 (사이킷런 방식으로 설명)   
- 100개의 결정트리를 훈련
-  

1. 부트스트랩 샘플 만들기
  - 부트스트랩 샘플: 예를들어 1000개 중에서 1개를 뽑고 다시 넣고 1개를 다시 뽑고 하는 방식으로 만들어진 샘플, 중복된 샘플을 추출가능하게 하기 위함, 부트스트랩 샘플은 훈련세트와 크기가 같다.

2. 일부 특성을 무작위로 고름
  - RandomForestClassifier : 전체 특성 개수의 제곱근만큼의 특성을 선택
  - RandomForestRegressor : 전체 특성을 사용
3. 이 특성 중에서 최선의 분할을 찾음
  - 노드만들기

회귀트리?  트리가 어떻게 회귀????    

 회귀 트리는 RSS(오차 제곱합)를 가장 잘 줄일 수 있는 변수(predictor)를 기준으로 분기(split)를 만들어 결과를 예측하는 매우 단순한 모델이다. 어떤 변수가 중요한지, 변수의 값에 따라 예측 결과가 무엇인지 한눈에 볼 수 있어 설명력이 좋은 장점이 있다.   
 


In [1]:
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier # 사이킷런1.0에서 히스토그램 기반 부스팅이 experimental 모듈 아래에서 벗어남

from sklearn.inspection import permutation_importance

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [2]:
pip list | grep scikit-learn  # 사이킷런 버전확인 1.2.1

scikit-learn                  1.2.1


In [3]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [9]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42, oob_score=True) 

rt_scores = cross_validate(rf, train_input, train_target, 
                       return_train_score=True, n_jobs=-1) # 교차검증수행, return_train_score매개변수 : 훈련세트에 대한 점수 반환
print('랜덤포레스트훈련셋점수:',np.mean(rt_scores['train_score']), 
                             '\n랜덤포레스트테스트셋점수:', np.mean(rt_scores['test_score']) )
print()

rf.fit(train_input, train_target)
print('특성중요도','alcohol, sugar, pH' ) 
print(rf.feature_importances_)             
# 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문에 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻음
print()


print('OOB score:',rf.oob_score_) # OOB(out of bag): 부트스트랩 샘플을 만들고 남은 샘플, OOB샘플을 이용하여 점수를 만듬 - 교차 검증을 대신할 수 있다

랜덤포레스트훈련셋점수: 0.9973541965122431 
랜덤포레스트테스트셋점수: 0.8905151032797809

특성중요도 alcohol, sugar, pH
[0.23167441 0.50039841 0.26792718]

OOB score: 0.8934000384837406


#### 트리의 앙상블 - 엑스트라 트리
- 랜덤포레스트와 비슷
- 다른점 : 부트스트랩샘플을 만들지 않고 전체 훈련 세트를 사용
- 노드를 분할할 때 무작위로 분할(?? 특성을 무작위로?? 노드를 무작위로 나눠서 만들면 성능이 터무니 없이 낮아지는 것 아닌가?)
- 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과

In [10]:
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
# 많은 트리를 학습해야하지만 랜덤하게 노드를 분할하기 때문에 계산 속도가 빠르다.

et_scores = cross_validate(et, train_input, train_target,
                        return_train_score = True, n_jobs=-1)
print('엑스트라트리트레인셋점수:',np.mean(et_scores['train_score']), '\n엑스트라트리테스트셋점수:',np.mean(et_scores['test_score']))
print()

et.fit(train_input, train_target)
print('특성중요도','alcohol, sugar, pH' ) 
print(et.feature_importances_)             # 당도에 대한 의존성이 낮아짐

엑스트라트리트레인셋점수: 0.9974503966084433 
엑스트라트리테스트셋점수: 0.8887848893166506

특성중요도 alcohol, sugar, pH
[0.20183568 0.52242907 0.27573525]


#### 트리의 앙상블 - 그레이디언트 부스팅
- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완 (sklearn의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정트리를 100개 사용)
- 깊이가 얕아서 과대적합에 강하고, 일반화 성능이 높다.
- 경사 하강법처럼 결정 트리를 계속 추가하면서 가장 낮은 곳(score가 좋은 곳)을 찾아 이동 

In [15]:
gb = GradientBoostingClassifier(random_state=42)
gb_scores = cross_validate(gb, train_input, train_target,
                        return_train_score = True, n_jobs=-1)
print('그레이디언트부스팅훈련셋점수:', np.mean(gb_scores['train_score']),'\n그레이디언트부스팅테스트셋점수:', np.mean(gb_scores['test_score']))
print()

# 매개변수 조정
gb2 = GradientBoostingClassifier(random_state=42, n_estimators = 500, learning_rate = 0.2) # learning_rate기본값 : 0.1
# n_estimators매개변수 : 결정트리개수, learning_rate매개변수 : 학습률 경사하강의 속도를 조절

gb2_scores = cross_validate(gb2, train_input, train_target,
                        return_train_score = True, n_jobs=-1)
print('매개변수조정훈련셋점수:', np.mean(gb2_scores['train_score']),'\n매개변수조정테스트셋점수:', np.mean(gb2_scores['test_score']))
print()

gb2.fit(train_input, train_target)
print('특성중요도','alcohol, sugar, pH' ) 
print(gb2.feature_importances_) 
# 순서대로 트리를 추가하기 때문에 느리지만 성능이 좋음

그레이디언트부스팅훈련셋점수: 0.8881086892152563 
그레이디언트부스팅테스트셋점수: 0.8720430147331015

매개변수조정훈련셋점수: 0.9464595437171814 
매개변수조정테스트셋점수: 0.8780082549788999

특성중요도 alcohol, sugar, pH
[0.15872278 0.68010884 0.16116839]


#### 트리의 앙상블 - 히스토그램 기반 그레이디언트 부스팅
- 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘
- 특성을 256개의 구간으로 나눔 (노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.)
- sklearn의 HistGradientBoostingClassifier에서는 기본 매개변수에서 안정적인 성능을 얻을 수 있다. (n_estimators 대신 max_iter을 사용)

In [18]:
hgb = HistGradientBoostingClassifier(random_state=42)

hgb_scores = cross_validate(hgb, train_input, train_target, return_train_score = True)
print('히스토그램기반그레이디언트부스팅훈련셋점수:', np.mean(hgb_scores['train_score']),'\n히스토그램기반그레이디언트부스팅테스트셋점수:', np.mean(hgb_scores['test_score']))
print()

# hgb에서 특성중요도 보기
# print(hgb.feature_importances_) hgb엔 이거 없음
# permutation_importance()함수 : 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지 계산
hgb.fit(train_input, train_target)
result_train = permutation_importance(hgb, train_input, train_target, 
                                n_repeats=10, random_state=42, n_jobs=-1)    # n_repeats매개변수 : 랜덤하게 섞을 횟수를 지정 (기본값=5)
print('훈련셋특성중요도','alcohol, sugar, pH' ) 
print(result_train.importances_mean) 
result_test = permutation_importance(hgb, test_input, test_target, 
                                n_repeats=10, random_state=42, n_jobs=-1)    # n_repeats매개변수 : 랜덤하게 섞을 횟수를 지정 (기본값=5)
print('테스트셋특성중요도','alcohol, sugar, pH' ) 
print(result_test.importances_mean) 
print()

print('테스트셋점수:',hgb.score(test_input, test_target))

히스토그램기반그레이디언트부스팅훈련셋점수: 0.9321723946453317 
히스토그램기반그레이디언트부스팅테스트셋점수: 0.8801241948619236

훈련셋특성중요도 alcohol, sugar, pH
[0.08876275 0.23438522 0.08027708]
테스트셋특성중요도 alcohol, sugar, pH
[0.05969231 0.20238462 0.049     ]

테스트셋점수: 0.8723076923076923


#### 사이킷런 이외에 그레이디언트 부스팅을 알고리즘을 구현한 라이브러리
1. XGBoost
2. LightGBM (빠르고 최신 기술이 많아서 인기가 많다)

In [21]:
xgb = XGBClassifier(tree_method = 'hist', random_state=42) # tree_method = 'hist' 히스토그램 기반 그레이디언트 부스팅 알고리즘으로 만듬
xgb_scores = cross_validate(xgb, train_input, train_target, 
                            return_train_score = True)
print('xgb훈련셋점수:', np.mean(xgb_scores['train_score']),'\nxgb테스트셋점수:', np.mean(xgb_scores['test_score']))
print()

lgb = LGBMClassifier(random_state=42)
lgb_scores = cross_validate(lgb, train_input, train_target, 
                            return_train_score = True)
print('lgb훈련셋점수:', np.mean(lgb_scores['train_score']),'\nlgb테스트셋점수:', np.mean(lgb_scores['test_score']))
print()

xgb훈련셋점수: 0.9555033709953124 
xgb테스트셋점수: 0.8799326275264677

lgb훈련셋점수: 0.9338079582727165 
lgb테스트셋점수: 0.8789710890649293

